# Building a Mobile App to Fight Lottery Addiction

Many people start playing the lottery for fun, but for some this activity turns into a habit which eventually escalates into addiction. Like other compulsive gamblers, lottery addicts soon begin spending from their savings and loans, they start to accumulate debts, and eventually engage in desperate behaviors like theft.

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. We are going to help this institute to create the logical core of the app and calculate probabilities.

For the first version of the app, we are going to focus on the 6/49 national lottery game in Canada to answer questions like:

- What is the probability of winning the big prize with a single ticket?
- What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
- What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

## Premise of the 6/49 Lottery Game

In the 6/49 lottery, 6 numbers are drawn from a set of 49 numbers ranging from 1 to 49. The drawing is done without replacement.

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. A player wins the big prize if the six numbers on their tickets match all the six numbers drawn. If a player has a ticket with the numbers {13, 22, 24, 27, 42, 44}, he only wins the big prize if the numbers drawn are {13, 22, 24, 27, 42, 44}. If only one number differs, he doesn't win.

## Core Functions

Throughout this project, we will need to calculate repeatedly probabilities and combinations. As a consequence, we will start by writing 2 functions to calculate factorials and calculate combinations.

In [17]:
def factorial(n):
    value = n
    for i in range(n-1,0,-1):
        value = value * i
    return value

def combinations(n,k):
    numerator = factorial(n)
    denominator = factorial(k) * factorial(n-k)
    return numerator/denominator

## Big Prize Win Probability

For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket.

We discussed with the engineering team of the medical institute, and they told us we need to be aware of the following details when we write the function:

- Inside the app, the user inputs six different numbers from 1 to 49.
- Under the hood, the six numbers will come as a Python list, which will serve as the single input to our function.
- The engineering team wants the function to print the probability value in a friendly way — in a way that people without any probability training are able to understand.

In [35]:
def one_ticket_probability(userTicket):
    if len(userTicket) != 6:
        return print('Error. Please submit 6 different numbers')
    total_outcomes = combinations(49,6)
    numSuccessful = 1
    probability = (1/total_outcomes)*100
    return print("You have a {prob:.7f}% chance to win the Big Prize with the ticket {tik}.".format(prob=probability,tik=userTicket))

one_ticket_probability([5,9,7,8,2,5])

You have a 0.0000072% chance to win the Big Prize with the ticket [5, 9, 7, 8, 2, 5].


## Comparing a Ticket to Historical Drawings

We just wrote a function to tell users the probability of winning the big prize for a single ticket. Users should also be able to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

We will focus on exploring the historical data coming from the Canada 6/49 lottery. The dataset is available on Kaggle. Lets read in the dataset and get familiar with it

In [45]:
import pandas as pd

lottery = pd.read_csv('649.csv')
print("Rows:    ",lottery.shape[0],'\nColumns: ',lottery.shape[1])

display(lottery.head(3))
display(lottery.tail(3))

Rows:     3665 
Columns:  11


,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34


,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


The numbers drawn are available in the columns "NUMBER DRAWN #", and there is also a bonus number drawn. The database contains the winning numbers every week from 6/12/1982 to 6/20/2018.

### Function to Compare User's Ticket to Historical Drawings

The engineering team told us that we need to be aware of the following details:

- Inside the app, the user inputs six different numbers from 1 to 49.
- Under the hood, the six numbers will come as a Python list and serve as an input to our function.
- The engineering team wants us to write a function that prints:
    - the number of times the combination selected occurred in the Canada data set; and
    - the probability of winning the big prize in the next drawing with that combination.
    
Lets create a function to compare a user ticket to historical drawings according to the specifications above.

In [75]:
def extract_numbers(row):
    row = row[4:10]
    row = set(row.values)
    return row

winningNumbers = lottery.apply(extract_numbers,axis=1)
winningNumbers.head(5)

def check_historical_occurence(userList,historical_num):
    userNumbers = set(userList)
    check_occurence = historical_num == userNumbers
    n_occurences = check_occurence.sum()
    
    if n_occurences == 0:
        print('''The combination {s} has never occured.
This doesn't mean it's more likely to occur now. Your chances to win the big prize in the next drawing using the combination {s} are 0.0000072%.
In other words, you have a 1 in {comb} chance to win'''.format(s=userNumbers,comb=combinations(49,6)))
    
    else:
        print('''The number of times the combination {s} has occured in the past is {occ}.
Your chances to win a big prize in the next drawing using the combination {s} are 0.0000072%.
In other words, you have a 1 in {comb} chance to win'''.format(s=userNumbers,occ = n_occurences,comb=combinations(49,6)))

In [78]:
check_historical_occurence([3,41,11,12,34,14],winningNumbers)

The combination {34, 3, 41, 11, 12, 14} has never occured.
This doesn't mean it's more likely to occur now. Your chances to win the big prize in the next drawing using the combination {34, 3, 41, 11, 12, 14} are 0.0000072%.
In other words, you have a 1 in 13983816.0 chance to win


## Calculating the Chances of Winning with Multiple Tickets

Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. 

In this section, we will write a function that will allow users to calculate the chances of winning for any number of tickets.

We've talked with the engineering team and they gave us the following information:

- The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
- Our function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
- The function should print information about the probability of winning the big prize depending on the number of different tickets played.


In [81]:
def multi_ticket_probability(n):
    combos = combinations(49,6)
    n_outcomes = n
    prob = (n_outcomes/combos)*100
    return print("If you purchase {n} different tickets for a single drawing, the probability of winning the big prize is {p:8f}%".format(n=n_outcomes,p=prob))

for i in [1,10,100,10000,1000000,6991908,13983816]:
    multi_ticket_probability(i)

If you purchase 1 different tickets for a single drawing, the probability of winning the big prize is 0.000007%
If you purchase 10 different tickets for a single drawing, the probability of winning the big prize is 0.000072%
If you purchase 100 different tickets for a single drawing, the probability of winning the big prize is 0.000715%
If you purchase 10000 different tickets for a single drawing, the probability of winning the big prize is 0.071511%
If you purchase 1000000 different tickets for a single drawing, the probability of winning the big prize is 7.151124%
If you purchase 6991908 different tickets for a single drawing, the probability of winning the big prize is 50.000000%
If you purchase 13983816 different tickets for a single drawing, the probability of winning the big prize is 100.000000%


## Calculating Probabilities for Getting Winning Numbers

In most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

These are the engineering details we'll need to be aware of:

Inside the app, the user inputs:
- six different numbers from 1 to 49; and
- an integer between 2 and 5 that represents the number of winning numbers expected
- Our function prints information about the probability of having the inputted number of winning numbers.

To calculate the probabilities, we tell the engineering team that the specific combination on the ticket is irrelevant and we only need the integer between 2 and 5 representing the number of winning numbers expected. Consequently, we will write a function named probability_less_6() which takes in an integer and prints information about the chances of winning depending on the value of that integer.

The function below calculates the probability that a player's ticket matches exactly the given number of winning numbers. If the player wants to find out the probability of having five winning numbers, the function will return the probability of having five winning numbers exactly (no more and no less). The function will not return the probability of having at least five winning numbers.

In [99]:
def probability_less_6(n):
    n_combinations_ticket = combinations(6,n)
    n_combinations_remaining = combinations(43,6-n)
    successful_outcomes = n_combinations_ticket * n_combinations_remaining
    
    total_poss_outcomes = combinations(49,6)
    prob = successful_outcomes/total_poss_outcomes
    
    probability_percentage = prob*100
    combinations_simplified = round(total_poss_outcomes/successful_outcomes)
    return print('''Your chances of having {} winning numbers with this ticket are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(n, probability_percentage,
                                                               int(combinations_simplified)))

Now, let's test the function on all the three possible inputs.

In [98]:
probability_less_6(4)

Your chances of having 4 winning numbers with this ticket are 0.096862%.
In other words, you have a 1 in 1,032 chances to win.


In [100]:
for test_input in [2, 3, 4, 5]:
    probability_less_6(test_input)
    print('--------------------------') # output delimiter

Your chances of having 2 winning numbers with this ticket are 13.237803%.
In other words, you have a 1 in 8 chances to win.
--------------------------
Your chances of having 3 winning numbers with this ticket are 1.765040%.
In other words, you have a 1 in 57 chances to win.
--------------------------
Your chances of having 4 winning numbers with this ticket are 0.096862%.
In other words, you have a 1 in 1,032 chances to win.
--------------------------
Your chances of having 5 winning numbers with this ticket are 0.001845%.
In other words, you have a 1 in 54,201 chances to win.
--------------------------


## Next steps

For the first version of the app, we coded four main functions:

- one_ticket_probability() — calculates the probability of winning the big prize with a single ticket
- check_historical_occurrence() — checks whether a certain combination has occurred in the Canada lottery data set
- multi_ticket_probability() — calculates the probability for any number of of tickets between 1 and 13,983,816
- probability_less_6() — calculates the probability of having two, three, four or five winning numbers exactly

Possible features for a second version of the app include:

- Making the outputs even easier to understand by adding fun analogies (for example, we can find probabilities for strange events and compare with the chances of winning in lottery; for instance, we can output something along the lines "You are 100 times more likely to be the victim of a shark attack than winning the lottery")
- Combining the one_ticket_probability() and check_historical_occurrence() to output information on probability and historical occurrence at the same time
- Create a function similar to probability_less_6() which calculates the probability of having at least two, three, four or five winning numbers.